<a href="https://colab.research.google.com/github/TallisHe/Automotive_Design_Copilot/blob/main/ADC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 环境安装及运行(打开执行我)


## **1. demo安装和运行**

说明:
  <br/>第一次执行: 1 -> 2 -> 3, 小标签2.1, 2.2按自己需求执行
  <br/>云盘启动(已缓存): 1 -> 3

In [ ]:
#@title ## 1、加载谷歌云盘并初始化环境(每次都必须执行)
from IPython.display import clear_output
import ipywidgets as widgets
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
#@markdown ####是否允许加载谷歌云盘（禁止将无法保存模型）：##
is_load_pan = "\u5141\u8BB8" #@param ["允许","禁止"]
import os
import concurrent.futures
run_state = False
mainpth = 'MyDrive'
def init_pan():
  if is_load_pan=="允许":
    from google.colab import drive
    drive.mount('/content/drive')
    run_state = True
  else:
    print('已禁止加载云盘')
    run_state = False
def glabdep_install():
  if not os.path.exists("/content/drive"):
    print('[1;31mGdrive not connected, using temporary colab storage ...')
    time.sleep(4)
    mainpth="MyDrive"
    !mkdir -p /content/drive/$mainpth
    Shared_Drive=""

  if Shared_Drive!="" and not os.path.exists("/content/drive/Shareddrives"):
    print('[1;31mShared drive not detected, using default MyDrive')
    mainpth="MyDrive"

  %mkdir -p /content/drive/$mainpth/sd

  %cd /content/
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/drive/'+mainpth+'/sd/stablediffusiond'): #restore later
    !tar -C /content/drive/$mainpth --zstd -xf sd_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  if not os.path.exists('drive/'+mainpth+'/sd/libtcmalloc/libtcmalloc_minimal.so.4'):
    %env CXXFLAGS=-std=c++14
    !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
    !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch
    %cd /content/gperftools
    !patch -p1 < /content/Patch
    !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
    !mkdir -p /content/drive/$mainpth/sd/libtcmalloc && cp .libs/libtcmalloc*.so* /content/drive/$mainpth/sd/libtcmalloc
    %env LD_PRELOAD=/content/drive/$mainpth/sd/libtcmalloc/libtcmalloc_minimal.so.4
    %cd /content
    !rm *.tar.gz Patch && rm -r /content/gperftools
  else:
    %env LD_PRELOAD=/content/drive/$mainpth/sd/libtcmalloc/libtcmalloc_minimal.so.4

  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py
def aira2_install():
  !apt -y update -qq
  # !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
  # %env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
  !apt-get -y install -qq aria2 libcairo2-dev pkg-config python3-dev
executor = concurrent.futures.ThreadPoolExecutor(max_workers=2)
init_pan()
# task1 = executor.submit(init_pan)
task2 = executor.submit(glabdep_install)
task3 = executor.submit(aira2_install)
# concurrent.futures.wait([task1,task2,task3])
concurrent.futures.wait([task2,task3])

clear_output()
# if run_state: print("云盘加载成功")
# else: print('已禁止加载云盘')
inf('\u2714 Done','success', '50px')

In [ ]:
#@title ## 2.0、下载各种插件(如果是云盘启动,执行第一次就可以)
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

sd0="s"+"t"+"a"+"b"+"l"+"e"+"-"+"d"+"i"+"f"+"f"+"u"+"s"+"i"+"o"+"n"+"-"+"w"+"e"+"b"+"u"
sd=sd0+'i'
server0="s"+"d"+"-"+"v"+"1"+"."+"5"+"."+"1"+"-"+"s"+"e"+"r"+"v"+"e"
server = server0+"r"
wi='w'+'e'+'b'+'u'+'i'
import os
#@markdown ####是否把SD服务和初始资源下载到网盘：##
saveInDrive = True # @param {type:'boolean'}
#@markdown ####初始化的大模型：##
first_checkpoint = "AnythingV5V3_v5PrtRE.safetensors" #@param ["Dark_sushi_mix.safetensors", "AnythingV5V3_v5PrtRE.safetensors", "chilloutmix_NiPrunedFp16Fix.safetensors", "rpg_V4.safetensors", "protogenV22Anime_22.safetensors","none"]
#@markdown ####stable diffusion的ui页面：##
sd_wi = "\u5B9A\u5236\u7248UI-Demo\u7248\u672C(base webui v1.6.0)" #@param ["定制版UI-Demo版本(base webui v1.6.0)","定制版UI-Demo版本(base webui v1.5.1)"]
#@markdown ####按定制版UI-Demo所需版本进行资源下载：##
is_customized_version = True # @param {type:'boolean'}
#@markdown ####是否加载云盘里的extensions、VAE、embeddings、lora、checkpoint？##
is_pan_extensions = True  # @param {type:'boolean'}
default_gdrive_path = "/content/drive/MyDrive" #@param {type: "string"}
base_dir = ""
if saveInDrive :
  os.makedirs(default_gdrive_path, exist_ok=True)
  os.makedirs(default_gdrive_path + "/sdwi", exist_ok=True)
  base_dir = default_gdrive_path + "/sdwi"
else:
  %cd /content/
  !mkdir /content/sdwi
  base_dir = "/content/sdwi"

with capture.capture_output() as cap:
  if sd_wi=="定制版UI-Demo版本(base webui v1.5.1)":
    # !git clone https://github.com/NullFullme/{server} --recursive {base_dir}
    %cd /content/
    !wget -q -i https://raw.githubusercontent.com/NullFullme/sd-v1.5.1-server/main/Dependencies/Custom-sd.txt
    if not os.path.exists(base_dir+'/repositories'):
      !tar -xvf "sd-v1.5.1-server.tar" --recursive-unlink -C {base_dir}
    # !dpkg -i *.deb
    # !tar -C / --zstd -xf gcolabdeps.tar.zst
    !rm *.deb | rm *.zst | rm *.txt
    !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py
  # elif sd_wi=="AUTOMATIC1111原版v1.3.0(稳定)":
  #   !git clone -b v1.3.0 --single-branch https://github.com/AUTOMATIC1111/{sd} {base_dir}
  # else:
  #   !git clone -b v1.5.1 --single-branch https://github.com/AUTOMATIC1111/{sd} {base_dir}
  elif sd_wi=="定制版UI-Demo版本(base webui v1.6.0)":
    %cd /content/
    !wget https://huggingface.co/riruA/dependencies/resolve/main/sd-v1.6.0-server.tar
    if not os.path.exists(base_dir+'/repositories'):
      !tar -xvf "sd-v1.6.0-server.tar" --recursive-unlink -C {base_dir}
    !rm *.deb | rm *.zst | rm *.txt
    !sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' /usr/lib/python3.10/warnings.py

!mkdir -p base_dir/cache/
os.environ['TRANSFORMERS_CACHE']=os.path.join(base_dir, "cache")
os.environ['TORCH_HOME'] = os.path.join(base_dir, "cache")

import concurrent.futures
import time
import subprocess

checkpoint_url = {
    "Dark_sushi_mix.safetensors"     : "https://huggingface.co/mdl-mirror/dark-sushi-mix/resolve/main/darkSushiMixMix_brighter.safetensors",
    "AnythingV5V3_v5PrtRE.safetensors"     : "https://huggingface.co/ckpt/anything-v5.0/resolve/main/AnythingV5V3_v5PrtRE.safetensors",
    "chilloutmix_NiPrunedFp16Fix.safetensors"      : "https://huggingface.co/naonovn/chilloutmix_NiPrunedFp32Fix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors",
    "rpg_V4.safetensors"         : "https://huggingface.co/Anashel/rpg/resolve/main/RPG-V4-Model-Download/RPG-v4.safetensors",
    "ProtoGen_X5.8-pruned-fp16.safetensors"         : "https://huggingface.co/darkstorm2150/Protogen_x5.8_Official_Release/resolve/main/ProtoGen_X5.8-pruned-fp16.safetensors",
    "none"         : "",
}

def run_git_download():
  start_time = time.time()
  !git clone https://github.com/Physton/sd-{wi}-prompt-all-in-one {base_dir}/extensions/sd-{wi}-prompt-all-in-one
  !git clone https://github.com/Mikubill/sd-{wi}-controlnet {base_dir}/extensions/sd-{wi}-controlnet
  !git -C {base_dir}/extensions/sd-{wi}-controlnet reset --hard 784eadbb
  !git clone https://github.com/fkunn1326/openpose-editor {base_dir}/extensions/openpose-editor
  !git clone https://github.com/DominikDoom/a1111-sd-{wi}-tagcomplete {base_dir}/extensions/a1111-sd-{wi}-tagcomplete
  !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 {base_dir}/extensions/ultimate-upscale
  !git clone https://github.com/hako-mikan/sd-{wi}-lora-block-weight {base_dir}/extensions/sd-{wi}-lora-block-weight
  #xformers加速
  end_time = time.time()
  print("已克隆git耗时：", end_time-start_time, "秒")

def run_aria2c_download():
  start_time = time.time()
  cmd=f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {checkpoint_url[first_checkpoint]}  -d {base_dir}/models/Stable-diffusion -o {first_checkpoint}"
  subprocess.run(cmd, shell=True)
  if is_customized_version:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth -d {base_dir}/models/ControlNet -o control_v11p_sd15_canny.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth -d {base_dir}/models/ControlNet -o control_v11p_sd15_inpaint.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth -d {base_dir}/models/ControlNet -o control_v11p_sd15_lineart.pth
  else:
    # 半精ControlNet
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11e_sd15_ip2p.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11f1p_sd15_depth.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15_canny.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15_inpaint.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11f1p_sd15_depth.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15_mlsd.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15_normalbae.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15_openpose.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15_scribble.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15_seg.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15_softedge.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11p_sd15s2_lineart_anime.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {base_dir}/models/ControlNet -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth -d {base_dir}/models/ESRGAN/ -o 4x-UltraSharp.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/daasd/CN.csv/resolve/main/CN.csv -d {base_dir}/extensions/a1111-sd-{wi}-tagcomplete/tags -o CN.csv
  end_time = time.time()
  print("aria2c完成下载耗时：", end_time-start_time, "秒")

def curl_download():
  start_time = time.time()
  !curl -Lo "{base_dir}/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors" https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
  !curl -Lo "{base_dir}/models/VAE/kl-f8-anime2.ckpt" https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/4c4f05104055c029ad577c18ac176462f0d1d7c1/vae/kl-f8-anime2.ckpt
  !curl -Lo "{base_dir}/models/VAE/animevae.pt" https://huggingface.co/swl-models/animvae/resolve/main/animevae.pt
  end_time = time.time()
  print("curl完成下载耗时：", end_time-start_time,"秒")

def wget_download():
  start_time = time.time()
  !apt install libunwind8-dev -yqq
  os.environ["LD_PRELOAD"] = "libtcmalloc.so.4"
  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  !sudo apt-get install sox ffmpeg libcairo2 libcairo2-dev
  end_time = time.time()
  print("wget完成下载耗时：", end_time-start_time,"秒")

def pip_download():
  start_time = time.time()
  !pip install xformers xformers==0.0.20
  end_time = time.time()
  print("pip完成下载耗时：", end_time-start_time,"秒")

with capture.capture_output() as cap:
  executor = concurrent.futures.ThreadPoolExecutor(max_workers=5)
  task1 = executor.submit(run_git_download)
  task2 = executor.submit(run_aria2c_download)
  task3 = executor.submit(curl_download)
  task4 = executor.submit(wget_download)
  task5 = executor.submit(pip_download)
  # concurrent.futures.wait([task1,task2,task3,task4,task5])
  concurrent.futures.wait([task1,task2,task3,task5])
  # concurrent.futures.wait([task5])



# with capture.capture_output() as cap:
#   %cd {base_dir}
#   !git config --unset core.hooksPath
#   import shutil
#   if os.path.exists(f'{base_dir}/embeddings'):
#     shutil.rmtree(f'{base_dir}/embeddings')
#   !git clone https://huggingface.co/nolanaatama/embeddings

#@markdown ####存放的路径如下(需要自己在网盘手动创建文件夹)：##
#@markdown #####extensions:  default_gdrive_path/extensions##
#@markdown #####VAE:  default_gdrive_path/VAE##
#@markdown #####embeddings:  default_gdrive_path/embeddings##
#@markdown #####lora:  default_gdrive_path/lora##
#@markdown #####checkpoint:  default_gdrive_path/checkpoint##

#个人插件从云盘的extensions文件夹与VAE文件夹加载
clear_output()
if is_pan_extensions:
  if os.path.exists(default_gdrive_path+"/extensions"):
    !rsync -a {default_gdrive_path}/extensions/* {base_dir}/extensions
    print('已加载云盘里的插件')
  if os.path.exists(default_gdrive_path+"/VAE"):
    !rsync -a {default_gdrive_path}/VAE/* {base_dir}/models/VAE
    print('已加载云盘里的VAE')
  if os.path.exists(default_gdrive_path+"/embeddings"):
    !rsync -a {default_gdrive_path}/embeddings/* {base_dir}/embeddings
    print('已加载云盘里的embeddings')
  if os.path.exists(default_gdrive_path+"/lora"):
    !mkdir -p {base_dir}/models/Lora
    !rsync -a {default_gdrive_path}/lora/* {base_dir}/models/Lora
    print('已加载云盘里的lora')
  if os.path.exists(default_gdrive_path+"/checkpoint"):
    !rsync -a {default_gdrive_path}/checkpoint/* {base_dir}/models/Stable-diffusion
    print('已加载云盘里的Stable-diffusion')


inf('\u2714 Done','success', '50px')

In [ ]:
#@title ## 2.1、下载列表管理/增删改查模型或其他文件
#@markdown ####选择mod集合(如果云盘没找到集合则在根目录创建)：##
mod_json_name = "model.json" #@param {type: "string"}
# 创建并写入文本到文件
mod_json_dir=''

#递归目录查找mod集合
def find_model_json(root_path,mod_json_name):
    for file_name in os.listdir(root_path):
        file_path = os.path.join(root_path, file_name)
        if os.path.isdir(file_path):
            res=find_model_json(file_path,mod_json_name)
            if res is not None:
              return res
        elif file_name == mod_json_name:
            print("找到model.json文件：", file_path)
            return file_path

if 'is_load_pan' in globals() and is_load_pan=="允许":
  mod_json_dir=find_model_json('/content/drive/',mod_json_name)
  if mod_json_dir==None:
    with open(f'/content/drive/MyDrive/{mod_json_name}', 'w') as f:
      f.write('[]')
      mod_json_dir=f'/content/drive/MyDrive/{mod_json_name}'
else:
  mod_json_dir=f'/content/{mod_json_name}'

import json
import pandas as pd
# drive.mount('/content/drive')
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from datetime import datetime
import time

custom_css = """
<style>
    .custom-button {
        padding: 10px 25px;
    }
    .custom-button.save {
        background-color: #67C23A;
        float:right
    }
    .custom-button.add {
        background-color: #409EFF;
    }
    .custom-button.delete {
        background-color: #F56C6C;
    }
    .custom-button.update {
        background-color: #E6A23C;
    }
</style>
"""
display(HTML(custom_css))

# 加载JSON数据为Python对象
try:
    # 尝试打开你的文件
    with open(mod_json_dir, 'r') as f:
        try:
            # 尝试使用json模块的load()函数，将文件内容转换为JSON
            data = json.load(f)
        except json.JSONDecodeError:
            # 如果出现 JSONDecodeError 异常，设置 data 为一个空字典
            data = {}
except FileNotFoundError:
    # 如果出现 FileNotFoundError 异常，设置 data 为一个空字典
    data = {}

print(mod_json_dir)
print(data)
# 将JSON数据转换为Pandas DataFrame
df = pd.DataFrame(data)

# 显示表格
def show_table():
    clear_output()
    # 新字段兼容
    if not df.columns.empty and 'path' not in df.columns:
      df['path'] = ''
    styled_df = df.style.set_properties(**{'text-align': 'center'})

    # 设置列宽度和居中对齐
    styled_df.set_table_styles([{'selector': 'th', 'props': [('max-width', '150px'), ('text-align', 'center')]}])

    # 输出DataFrame
    display(styled_df)
    display(widgets.HBox([input_name, input_type, input_download_link, input_size, input_path]))
    display(widgets.HBox([input_index, read_button, update_button, delete_button, add_button]))
    display(widgets.HBox([save_button]))

# 添加一行数据
def add_row(button):
  try:
    if not input_download_link.value:
      display(HTML(f"<p style='color: red;'>下载链接为必填项</p>"))
      return

    new_row = {
        "name": input_name.value.strip() or '未命名',
        "type": '未定义' if input_type.value == '不选则根据下载内容决定' else input_type.value,
        "downloadLink": input_download_link.value.strip(), # 修改这里的键名
        "size": input_size.value,
        "path": input_path.value,
        "lastChangeDate": datetime.now().strftime('%Y-%m-%d')
    }
    global df
    df = df.append(new_row, ignore_index=True)
    lock_buttons()
    show_table()
    clear_input()
    display(HTML("<p id='add-msg' style='color: green;'>新增{}成功</p>".format(new_row)))
    time.sleep(1)
    display(HTML("<script>document.getElementById('add-msg').remove()</script>"))
  except Exception as e:
    display(HTML(f"<p style='color: red;'>新增出错：{e}</p>"))

# 删除一行数据
def delete_row(button):
  try:
    index = int(input_index.value)
    global df
    df = df.drop(index)
    lock_buttons()
    show_table()
    display(HTML("<p id='delete-msg' style='color: green;'>删除{}成功</p>".format(index)))
    time.sleep(1)
    display(HTML("<script>document.getElementById('delete-msg').remove()</script>"))
  except Exception as e:
    display(HTML(f"<p style='color: red;'>删除出错：{e}</p>"))

#读取一行数据
def read_row(button):
    try:
        index = int(input_index.value)
        input_name.value = df.at[index, 'name']
        input_type.value ='不选则根据下载内容决定' if df.at[index, 'type']=='未定义' else df.at[index, 'type']
        input_download_link.value = df.at[index, 'downloadLink']
        input_size.value = df.at[index, 'size']
        input_path.value = df.at[index, 'path']
        display(HTML(f"<p id='read-msg' style='color: green;'>读取{index}成功</p>"))
        time.sleep(1)
        display(HTML("<script>document.getElementById('read-msg').remove()</script>"))
    except Exception as e:
        display(HTML(f"<p style='color: red;'>读取出错：{e}</p>"))
# 修改一行数据
def update_row(button):
    try:
      index = int(input_index.value)
      if input_download_link.value:
        df.at[index, 'downloadLink'] = input_download_link.value.strip()
      else:
        display(HTML(f"<p style='color: red;'>下载链接为必填项{e}</p>"))
        return
      df.at[index, 'name'] = input_name.value.strip() or '未命名'
      df.at[index, 'type'] = '未定义' if input_type.value == '不选则根据下载内容决定' else input_type.value
      df.at[index, 'size'] = input_size.value
      df.at[index, 'path'] = input_path.value or ''
      df.at[index, 'lastChangeDate'] = datetime.now().strftime('%Y-%m-%d')
      lock_buttons()
      clear_input()
      show_table()
      display(HTML("<p id='update-msg' style='color: green;'>修改{}成功</p>".format(index)))
      time.sleep(1)
      display(HTML("<script>document.getElementById('update-msg').remove()</script>"))
    except Exception as e:
      display(HTML(f"<p style='color: red;'>修改出错：{e}</p>"))

#清空输入框
def clear_input():
    try:
        input_name.value = ''
        input_type.value = ''
        input_download_link.value = ''
        input_size.value = ''
        input_path.value = ''
    except Exception as e:
        display(HTML(f"<p style='color: red;'>重置输入框出错：{e}</p>"))

#保存
def save_row(button):
    global df
    try:
        with open(mod_json_dir, 'w') as Wfile:
            json=df.to_json(orient='records')
            Wfile.write(json)
            display(HTML("<p id='save-msg' style='color: green;'>保存成功</p>"))
            time.sleep(1)
            display(HTML("<script>document.getElementById('save-msg').remove()</script>"))
    except Exception as e:
        display(HTML(f"<p style='color: red;'>保存出错：{e}</p>"))



# 定义输入框和按钮
input_name = widgets.Text(description='名字',placeholder='不填则根据下载内容决定')
input_type = widgets.Text(description='类型',placeholder='选填')
input_download_link = widgets.Text(description='下载链接',placeholder='必填：C站或者huggingface') # 修改这里的描述
input_size = widgets.Text(description='大小',placeholder='选填')
input_path = widgets.Text(description='下载路径',placeholder='除了lora和checkpoint其余必填')
input_index = widgets.Text(description='mod编号',placeholder='与新增无关，用于读改删mod',layout=widgets.Layout(margin='10px'))
add_button = widgets.Button(description='新增mod', layout=widgets.Layout(margin='10px 180px'), button_style='info')
delete_button = widgets.Button(description='删除mod', layout=widgets.Layout(margin='10px 10px'), button_style='danger')
update_button = widgets.Button(description='更新mod', layout=widgets.Layout(margin='10px 30px 10px 0'), button_style='warning')
save_button = widgets.Button(description='保存', layout=widgets.Layout(margin='20px auto'), button_style='success')
read_button = widgets.Button(description='读取mod', layout=widgets.Layout(margin='10px 0px 10px 30px'), button_style='primary')


#节流
def lock_buttons():
    # add_button.disabled = True
    delete_button.disabled = True
    update_button.disabled = True

def unlock_buttons():
    # add_button.disabled = False
    delete_button.disabled = False
    update_button.disabled = False

def on_input_change(change):
  if change.new:
    # len(change.new)
    if change.new==change.owner.value:
      if len(change.new)==1:
        lock_buttons()
        time.sleep(0.5)
        unlock_buttons()
  else:
    lock_buttons()


# 为输入框添加输入事件
# input_name.observe(on_input_change, names='value')
# input_type.observe(on_input_change, names='value')
# input_download_link.observe(on_input_change, names='value')
# input_size.observe(on_input_change, names='value')
input_index.observe(on_input_change, names='value')

add_button.on_click(add_row)
delete_button.on_click(delete_row)
update_button.on_click(update_row)
save_button.on_click(save_row)
read_button.on_click(read_row)

# 显示表格和交互式按钮
lock_buttons()
show_table()

In [ ]:
#@title ## 2.2、按照2.1的列表开始下载
#@markdown ####是否下载到网盘：##
saveInDrive = True # @param {type:'boolean'}
default_gdrive_path = "/content/drive/MyDrive" #@param {type: "string"}
base_dir = ""
if saveInDrive :
  base_dir = default_gdrive_path
else:
  base_dir = "/content/sdwi"
#@markdown ####（可选）只进行部分下载就填（复制名字按“与“字分割）：##
mod_name = "" #@param {type: "string"}
sd0="s"+"t"+"a"+"b"+"l"+"e"+"-"+"d"+"i"+"f"+"f"+"u"+"s"+"i"+"o"+"n"+"-"+"w"+"e"+"b"+"u"
sd=sd0+'i'
check_dir=""
lora_dir=""
if saveInDrive:
  check_dir=f(base_dir+"/checkpoint")
  lora_dir=f(base_dir+"/lora")
else:
  check_dir=f(base_dir+"/models/Stable-diffusion")
  lora_dir=f(base_dir+"/models/Lora")

import json
import re
import subprocess
import shutil
import os
from concurrent.futures import ThreadPoolExecutor

def swap(arr,l,r):
  temp=arr[l]
  arr[l]=arr[r]
  arr[r]=temp

def checkRelateMod(oldMod, newMod):
  try:
    return oldMod and newMod and oldMod['downloadLink'] == newMod['downloadLink']
  except Exception as e:
    print(e)


def sanitize_filename(filename):
    # 定义不兼容字符的正则表达式模式
    pattern = r'[\\/:*?"<>|]'
    # 使用下划线替换不兼容字符
    sanitized_filename = re.sub(pattern, '_', filename)
    return sanitized_filename

def dealRelateMod(oldRenameMod, newRenameMod):
  old_file_path = f"{oldRenameMod['path']}/{oldRenameMod['name']}"
  #新模型的定义不带后缀名，所以需要加上处理，且在预处理情况下不知道路径那么只能沿用老模型的路径和名字
  new_file_dir= (
    newRenameMod['path']
    if newRenameMod.get('path') and newRenameMod['path'] != ""
    else oldRenameMod['path']
    )
  new_file_name=(
      sanitize_filename(newRenameMod['name'])+os.path.splitext(oldRenameMod['name'])[1]
      if newRenameMod['name'] and newRenameMod['name'] !="未命名"
      else oldRenameMod['name']
    )
  new_file_path = f"{new_file_dir}/{new_file_name}"
  if old_file_path != new_file_path:
    try:
        if os.path.exists(old_file_path):
            os.rename(old_file_path,new_file_path)
            oldRenameMod['path']=new_file_dir
            oldRenameMod['name']=new_file_name
            print(f"{old_file_path}更改为{new_file_path}成功")
        else:
            print(f"{old_file_path}不存在，重命名失败")
    except OSError as e:
        print(f"更改{old_file_path}时发生错误: {e}")
  else:
    print(f"{newRenameMod['name']}已存在且无任何改变，将忽视")

def deleteMod(oldDeMod):
  file_path = f"{oldDeMod['path']}/{oldDeMod['name']}"
  try:
      if os.path.exists(file_path):
          os.remove(file_path)
          #oldCo的行也要进行同步删除
          print(f"{oldDeMod['name']}删除成功")
          del oldDeMod
      else:
          print(f"{oldDeMod['name']}不存在，删除失败")
  except OSError as e:
      print(f"删除{oldDeMod['name']}时发生错误: {e}")

def LCS(oldCo, newCo):
  oldStartIdx = 0   # 旧前指针
  newStartIdx = 0   # 新前指针
  oldEndIdx = len(oldCo) - 1  # 旧后指针
  newEndIdx = len(newCo) - 1  # 新后指针
  oldStartMod = oldCo[oldStartIdx]  # 旧前Mod
  oldEndMod = oldCo[oldEndIdx]  # 旧后Mod
  newStartMod = newCo[newStartIdx]  # 新前Mod
  newEndMod = newCo[newEndIdx]  # 新后Mod
  linkMap = {}
  while oldStartIdx <= oldEndIdx and newStartIdx <= newEndIdx:
    # 新前旧前
    if checkRelateMod(oldStartMod, newStartMod):
      dealRelateMod(oldStartMod, newStartMod)
      oldStartIdx += 1
      newStartIdx += 1
      if newStartIdx > newEndIdx or oldStartIdx > oldEndIdx:
        break
      oldStartMod = oldCo[oldStartIdx]
      newStartMod = newCo[newStartIdx]
    # 新后旧后
    elif checkRelateMod(oldEndMod, newEndMod):
      dealRelateMod(oldEndMod, newEndMod)
      oldEndIdx -= 1
      newEndIdx -= 1
      if newStartIdx > newEndIdx or oldStartIdx > oldEndIdx:
        break
      oldEndMod = oldCo[oldEndIdx]
      newEndMod = newCo[newEndIdx]
    # 新后旧前
    elif checkRelateMod(oldStartMod, newEndMod):
      dealRelateMod(oldStartMod, newEndMod)
      oldStartIdx += 1
      newEndIdx -= 1
      if newStartIdx > newEndIdx or oldStartIdx > oldEndIdx:
        break
      oldStartMod = oldCo[oldStartIdx]
      newEndMod = newCo[newEndIdx]
    # 新前旧后
    elif checkRelateMod(oldEndMod, newStartMod):
      dealRelateMod(oldEndMod, newStartMod)
      oldEndIdx -= 1
      newStartIdx += 1
      if newStartIdx > newEndIdx or oldStartIdx > oldEndIdx:
        break
      oldEndMod = oldCo[oldEndIdx]
      newStartMod = newCo[newStartIdx]
    # 四种均未找到
    else:
      # if not linkMap:
      linkMap = {}
    # 从 oldStartIdx 开始，到oldEndIdx结束，创建linkMap映射对象
      for i in range(oldStartIdx, oldEndIdx+1):
        downloadLink = oldCo[i]['downloadLink']
        if downloadLink is not None:
          linkMap[downloadLink] = i
      idxInOld = linkMap.get(newStartMod['downloadLink'], None)
      if idxInOld is None:
        print(f"{newStartMod['name']}不存在于旧mod,{newStartMod['downloadLink']}将添加到下载任务队列")
        content.append(newStartMod)

      else:
        dealRelateMod(oldCo[idxInOld], newStartMod)
        swap(oldCo,idxInOld,oldEndIdx)
        oldEndIdx-=1
      # 指针下移，移动新的头
      newStartIdx += 1
      if newStartIdx > newEndIdx:
          break
      newStartMod = newCo[newStartIdx]
      oldEndMod=oldCo[oldEndIdx]
      # oldStartIdx += 1
  print('new',newStartIdx,newEndIdx)
  print('old',oldStartIdx,oldEndIdx)
  # new这里还有剩余Mod没有处理
  if newStartIdx <= newEndIdx:
    for i in range(newStartIdx, newEndIdx + 1):
      print(f"{newCo[i]['name']}为新且不重复mod，将添加到下载任务队列")
      content.append(newCo[i])

  elif oldStartIdx <= oldEndIdx:
    for i in range(oldStartIdx, oldEndIdx + 1):
      if oldCo[i]:
        print(f"{oldCo[i]['name']}为旧不重复mod，将删除")
        deleteMod(oldCo[i])

# LCS(oldArr,oldArr)
with open(mod_json_dir, 'r') as modelFile:
  preContent=json.loads(modelFile.read())

#更新阶段
if 'oldCo' in globals() and len(oldCo)>0:
  content=[]
  print(oldCo)
  print(preContent)
  LCS(oldCo, preContent)
  print(content)
#初始化阶段
else:
  oldCo=[]
  content=preContent


#部分下载的情况
selected_mods = [x.strip() for x in mod_name.split('与') if x.strip()]




def get_file_size(file_path):
    try:
        file_size_bytes = os.path.getsize(file_path)
        for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
            if file_size_bytes < 1024.0:
                break
            file_size_bytes /= 1024.0
        return f"{file_size_bytes:.2f} {unit}"
    except OSError as e:
        print(f"Error: {e}")
        return None

def get_civitai_file(str):
    match = re.search(r'[^/]+$', str.decode())
    if match:
      return match.group(0).split('\n')[0]
    else:
      print("无法从输出中提取文件名")

def move_model(source_path, target_path):
    try:
        shutil.move(source_path, target_path)
    except OSError as e:
        print(f"Error: {e}")

def download_file(item):
  # for item in content:
    if selected_mods and item['name'] not in selected_mods:
        return
    download_url=item['downloadLink']
    #huggingface
    match = re.search(r'/([^/]*)$', download_url)
    file_name = match.group(1)
    #开始下载
    cmd = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_url} -d {lora_dir}" if '.' not in match.group(1) else f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_url} -d {lora_dir} -o {file_name}"
    result = subprocess.run(cmd, shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    if result.returncode == 0:
      # 判断是civitai还是hugging face还是自定义
      #civitai
      if '.' not in file_name:
        file_name =get_civitai_file(result.stdout)
      #如果定义了名字则重命名
      if item['name']!='未命名':
        #需要进行文件名的兼容处理
        temp_name=sanitize_filename(item['name'])+os.path.splitext(file_name)[1]
        os.rename(f"{lora_dir}/{file_name}",f"{lora_dir}/{temp_name}")
        final_name =temp_name
      else:
        final_name =file_name
      source_path = f"{lora_dir}/{final_name}"
      file_size = get_file_size(source_path)
      #如果path定义了则进行移动
      if item.get('path') and item['path'] != "":
          target_path = item['path']
          move_model(source_path, target_path)
          item['path']=target_path
          print(file_name+'已下载，重命名为：'+final_name)
          print('移动--',final_name,f'到{target_path}')
      #如果为checkpoint则进行移动
      elif file_size and 'GB'in file_size:
          move_model(source_path, check_dir)
          item['path']=check_dir
          print(file_name+'已下载，重命名为：'+final_name)
          print('移动checkpoint--',final_name,f'到{check_dir}文件夹')
      #如果为lora则直接调用source_path
      else:
          item['path']=lora_dir
          print(file_name+'已下载，重命名为：'+final_name)
      #改变item['name']用于最小化更新的重命名判断
      item['name']=final_name
      oldCo.append(item)
    else:
      print(f"{item['name']}下载失败,请检查{item['downloadLink']}")

# download_file(item)
# 使用线程池进行下载
with ThreadPoolExecutor(max_workers=5) as executor:
    for item in content:
        executor.submit(download_file, item)

In [ ]:
#@title # **3、运行/重启**
from IPython.utils import capture
from IPython.display import clear_output
import os
sd0="s"+"t"+"a"+"b"+"l"+"e"+"-"+"d"+"i"+"f"+"f"+"u"+"s"+"i"+"o"+"n"+"-"+"w"+"e"+"b"+"u"
sd=sd0+'i'
#@markdown ####从云盘启动：##
startInDrive = True # @param {type:'boolean'}
default_gdrive_path = "/content/drive/MyDrive" #@param {type: "string"}
base_dir = ""
if startInDrive :
  base_dir = default_gdrive_path + "/sdwi"
else:
  base_dir = "/content/sdwi"
%cd {base_dir}
# outputs_dir = base_dir + "/outputs"
# if os.path.exists(outputs_dir):
#   pass
# else:
#   !mkdir outputs
#   print("创建outputs目录")
os.environ['TRANSFORMERS_CACHE']=os.path.join(base_dir, "cache")
os.environ['TORCH_HOME'] = os.path.join(base_dir, "cache")

# with capture.capture_output() as cap:
#   !git reset --hard
#   !git checkout main
#   time.sleep(1)
#   !rm webui.sh
#   !git pull
# clear_output()
#@markdown ####启动定制版Demo：##
is_Demo = True  # @param {type:'boolean'}
#@markdown ####全精度/半精度启动：##
is_full_precision = True  # @param {type:'boolean'}
#@markdown #(重要!! **ngrok** 必须获取,第一次使用先注册,然后邮件验证帐号再获取token)
#@markdown #### 验证后填入一次即可
#@markdown ####(获取[ngrok](https://dashboard.ngrok.com/get-started/your-authtoken)的token进行免费网络加速：##
ngrok_auth="2V9HDEfPlGGfJTb2b8s334hhOc2_oA8jroor1ZN6Sw9FM6Fu"  #@param {type:"string"}

# !git -C base_dir/repositories/k-diffusion reset --hard HEAD

!pip uninstall torch -y
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

if ngrok_auth == "":
  print("请获取token后重新运行")
else:
  if is_Demo:
    full_precision_str="--nowebui --port 7789 --share --lowram --disable-safe-unpickle  --disable-console-progressbars --xformers --enable-insecure-extension-access --precision full --no-half --no-half-vae --opt-sub-quad-attention --opt-channelslast --api"
    half_precision_str="--nowebui --port 7789 --share --lowram  --disable-safe-unpickle  --disable-console-progressbars --xformers --enable-insecure-extension-access  --opt-sub-quad-attention --opt-channelslast --api"

    if ngrok_auth:
      full_precision_str+=f"  --ngrok={ngrok_auth} --ngrok-region='auto'"
      half_precision_str+=f"  --ngrok={ngrok_auth} --ngrok-region='auto'"
    if is_full_precision:
      !python launch.py {full_precision_str} #（解决精度足但速度不够）
    else:
      !pyt
  else:
    full_precision_str="--share --lowram --disable-safe-unpickle  --disable-console-progressbars --xformers --enable-insecure-extension-access --precision full --no-half --no-half-vae --opt-sub-quad-attention --opt-channelslast --api"
    half_precision_str="--share --lowram  --disable-safe-unpickle  --disable-console-progressbars --xformers --enable-insecure-extension-access  --opt-sub-quad-attention --opt-channelslast --api"

    if ngrok_auth:
      full_precision_str+=f"  --ngrok={ngrok_auth} --ngrok-region='auto'"
      half_precision_str+=f"  --ngrok={ngrok_auth} --ngrok-region='auto'"
    if is_full_precision:
      !python launch.py {full_precision_str} #（解决精度足但速度不够）
    else:
      !python launch.py {half_precision_str} #半精度（速度提升1倍以上，但可能出现精度不足问题）

In [ ]:
#@title ## 其他自定义下载
#@markdown ####下载地址：##
download_url = "" #@param {type: "string"}
#@markdown ####存放目录：##
download_dir = "" #@param {type: "string"}
#@markdown ####文件名：##
download_name = "" #@param {type: "string"}

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_url} -d {download_dir} -o {download_name}


In [ ]:
#@title ## 删除临时硬盘的SD，重新安装和部署（别乱点）(用于临时部署模式非网盘模式)
sd0="s"+"t"+"a"+"b"+"l"+"e"+"-"+"d"+"i"+"f"+"f"+"u"+"s"+"i"+"o"+"n"+"-"+"w"+"e"+"b"+"u"
sd=sd0+"i"
import os
import shutil
import ipywidgets as widgets
import time
from IPython.display import display
from google.colab import output

def delete_lora_folder(b):
    folder_path = f'/content/sdwi'
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print("已成功删除，请重新安装")
        time.sleep(5)
        output.clear()
    else:
        print("已取消删除")
        time.sleep(5)
        output.clear()

def on_button_click(button):
    delete_lora_folder(button)

button = widgets.Button(description="确定要删除吗?")
button.on_click(on_button_click)
display(button)


In [ ]:

#@title 查看GPU和VRAM可用信息

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# 模型下载

In [ ]:
#@title Checkpoints/safetensors等大模型下载
a="stable"
b="Stable"
url = "https://civitai.com/api/download/models/128713" #@param {type:"string"}
#@ markdown - _Remote source url._
destination_folder = "/content/drive/MyDrive/checkpoint" #@param {type:"string"}
#@ markdown - _Find folder in gdrive on the left and right-click to copy the path._
filename = "DreamShaper.safetensors" #@param {type:"string"}
#@ markdown - _Filename with extension._

target = f'{destination_folder}/{filename}'

!gdown -O $target "$url"


In [ ]:
#@title Lora模型下载
a="stable"
url = "https://civitai.com/api/download/models/150273" #@param {type:"string"}
#@ markdown - _Remote source url._
destination_folder = "/content/drive/MyDrive/lora" #@param {type:"string"}
#@ markdown - _Find folder in gdrive on the left and right-click to copy the path._
filename = "Genshin.safetensors" #@param {type:"string"}
#@ markdown - _Filename with extension._

target = f'{destination_folder}/{filename}'

!gdown --fuzzy -O $target "$url"

In [ ]:
#@title embeddings模型下载
a="stable"
url = "https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors" #@param {type:"string"}
#@ markdown - _Remote source url._
destination_folder = "/content/drive/MyDrive/embeddings" #@param {type:"string"}
#@ markdown - _Find folder in gdrive on the left and right-click to copy the path._
filename = "EasyNegative.safetensors" #@param {type:"string"}
#@ markdown - _Filename with extension._

target = f'{destination_folder}/{filename}'

!gdown --fuzzy -O $target "$url"

In [ ]:

#@title VAE下载
a="stable"
url = "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors" #@param {type:"string"}
#@ markdown - _Remote source url._
destination_folder = "/content/drive/MyDrive/VAE" #@param {type:"string"}
#@ markdown - _Find folder in gdrive on the left and right-click to copy the path._
filename = "vae-ft-mse-840000-ema-pruned.safetensors" #@param {type:"string"}
#@ markdown - _Filename with extension._

target = f'{destination_folder}/{filename}'

!gdown --fuzzy -O $target "$url"

In [ ]:
#@title 自定义下载
url = "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors" #@param {type:"string"}
#@markdown - _下载链接._
destination_folder = "/content/drive/MyDrive/models" #@param {type:"string"}
#@markdown - _在google云盘中找到想要下载的目标路径右键复制地址._
filename = "vae-ft-mse-840000-ema-pruned.safetensors" #@param {type:"string"}
#@markdown - _保存的文件名._

target = f'{destination}/{filename}'

!gdown --fuzzy -O $target "$url"

# 生成图片下载到本地

In [ ]:
#@title 从云盘下载生成图片到本地
from google.colab import files
%ls
a="stable"
!zip -r /content/outputs.zip /content/drive/MyDrive/sdwi/outputs
files.download('/content/outputs.zip')

# **安装调试服务环境**

In [ ]:
#@title ## 1、调试环境
import random, string, urllib.request, json, getpass

#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')

#Print root password
print(f'Root password: {password}')


In [ ]:
#@title ## 2、安装SSH服务
! apt-get install openssh-server
! mkdir /var/run/sshd
! echo "root:root" | chpasswd
! sed -i 's/PermitRootLogin prohibit-password/PermitRootLogin yes/' /etc/ssh/sshd_config
! service ssh restart

In [ ]:
#@title ## 3、获取链接信息
import socket
print(socket.gethostbyname(socket.gethostname()))